In [1]:
import pandas as pd
import glob
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline 


# Seaborn visualization library
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier

import scipy

In [2]:
pd.set_option('display.max_columns', 1000)
pd.options.display.max_rows=1000

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

plt.rcParams.update({'figure.max_open_warning': 0})


In [3]:
import random
random.seed( 0 )

# Load in Data

In [4]:
root = r"../data/input/07 Samsung UX Index - Web App Implementation/"

fname_data = root + r"Samsung UX Index Survey_Data.csv"
df_data = pd.read_csv(fname_data)

fname_vaxmap = root + r"Samsung UX Index Survey_Datamap.xlsx"
df_varmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=0)
df_valmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=1)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1218) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Examine ATTRIBUTE Importance (Zclass)

In [11]:
path = r'/Users/lubagloukhov/Documents/Consulting/Samsung/UXi/data/output/20191217_SegUsers'
all_files = glob.glob(path + "*/*/Seg1_KNN3_zclust0.05.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['analysisloc'] = filename.split('/')[10]
    li.append(df)

zclust_frame = pd.concat(li, axis=0, ignore_index=True)

print(zclust_frame.shape)
zclust_frame.head()

(5220, 8)


,Variable,Label,interp,clusterA,clusterB,stat,pvalue,analysisloc
0,d3_2,D3. Employed - Which of the following best describes your current employment status?,NaN,0,0,0.000000,1.000000,20191217_091942_activitiesximportancexallATTR
1,d3_2,D3. Employed - Which of the following best describes your current employment status?,Accept H0: cluster 0 >= cluster 1,0,1,-0.110387,0.912122,20191217_091942_activitiesximportancexallATTR
2,d3_2,D3. Employed - Which of the following best describes your current employment status?,Accept H0: cluster 0 <= cluster 2,0,2,0.802636,0.422358,20191217_091942_activitiesximportancexallATTR
3,d3_2,D3. Employed - Which of the following best describes your current employment status?,Accept H0: cluster 1 <= cluster 0,1,0,0.110387,0.912122,20191217_091942_activitiesximportancexallATTR
4,d3_2,D3. Employed - Which of the following best describes your current employment status?,NaN,1,1,0.000000,1.000000,20191217_091942_activitiesximportancexallATTR


In [12]:
frame_Accept005 = zclust_frame[zclust_frame.interp.fillna(value='').str.contains('Reject')]

print(frame_Accept005.shape)
print(float(frame_Accept005.shape[0]/2484))
frame_Accept005.head()

(1392, 8)
0.5603864734299517


,Variable,Label,interp,clusterA,clusterB,stat,pvalue,analysisloc
37,d7_1,D7. White - What is your race?,Reject H0 in favor of Ha: cluster 0 < cluster 1,0,1,-4.542998,6.212645e-06,20191217_091942_activitiesximportancexallATTR
39,d7_1,D7. White - What is your race?,Reject H0 in favor of Ha: cluster 1 > cluster 0,1,0,4.542998,6.212645e-06,20191217_091942_activitiesximportancexallATTR
41,d7_1,D7. White - What is your race?,Reject H0 in favor of Ha: cluster 1 > cluster 2,1,2,7.763984,1.164783e-14,20191217_091942_activitiesximportancexallATTR
43,d7_1,D7. White - What is your race?,Reject H0 in favor of Ha: cluster 2 < cluster 1,2,1,-7.763984,1.164783e-14,20191217_091942_activitiesximportancexallATTR
46,d7_2,D7. Black or African American - What is your race?,Reject H0 in favor of Ha: cluster 0 > cluster 1,0,1,3.703290,2.256509e-04,20191217_091942_activitiesximportancexallATTR


In [13]:
frame_Accept005.groupby('Variable').Label.count().sort_values()

Variable
d3_1_0                      0
d7_97                       2
d7_4                        2
d4_3                        2
d6                          4
d4_7                        4
d3_2                        6
d4_5                        6
d4_4                        8
qxcurrentxos_1              8
d3_3                        8
qxactivitiesxrecency_4     10
d1_3                       10
qxcurrentxos_6             12
d7_2                       14
d7_99                      16
d4_2                       16
d4_1                       18
qxactivitiesxrecency_8     22
d4_6                       22
qxactivitiesxrecency_5     24
qxcurrentxos_8             24
qxactivitiesxrecency_2     26
qxactivitiesxrecency_22    26
qxactivitiesxrecency_20    26
qxactivitiesxrecency_12    30
qxactivitiesxrecency_13    30
d3_1_1                     30
qxactivitiesxrecency_6     32
d7_1                       32
qxcurrentxos_2             32
qxactivitiesxrecency_7     34
qxcurrentxos_4             34
q

In [14]:
frame_Accept001 = frame_Accept005[frame_Accept005.pvalue<.000000001]
# frame[frame.interp.fillna(value='').str.contains('Reject')]

print(frame_Accept001.shape)
print(float(frame_Accept001.shape[0]/2484))
frame_Accept001.head()

(272, 8)
0.10950080515297907


,Variable,Label,interp,clusterA,clusterB,stat,pvalue,analysisloc
41,d7_1,D7. White - What is your race?,Reject H0 in favor of Ha: cluster 1 > cluster 2,1,2,7.763984,1.164783e-14,20191217_091942_activitiesximportancexallATTR
43,d7_1,D7. White - What is your race?,Reject H0 in favor of Ha: cluster 2 < cluster 1,2,1,-7.763984,1.164783e-14,20191217_091942_activitiesximportancexallATTR
50,d7_2,D7. Black or African American - What is your race?,Reject H0 in favor of Ha: cluster 1 < cluster 2,1,2,-6.630354,4.077261e-11,20191217_091942_activitiesximportancexallATTR
52,d7_2,D7. Black or African American - What is your race?,Reject H0 in favor of Ha: cluster 2 > cluster 1,2,1,6.630354,4.077261e-11,20191217_091942_activitiesximportancexallATTR
100,qxactivitiesxrecency_1,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 0 > cluster 1,0,1,6.411188,2.379396e-10,20191217_091942_activitiesximportancexallATTR


In [15]:
print(len(frame_Accept001.groupby('analysisloc').Variable.count()))
frame_Accept001.groupby('analysisloc').Variable.count().sort_values()


9


analysisloc
20191217_091917_loyaltymetricsxallATTR              6
20191217_092034_upgradetransxallATTR               10
20191217_091909_brandlovetrustxallATTR             12
20191217_092005_activitiesxsatisfactionxallATTR    12
20191217_091952_activitiexsqualityxallATTR         18
20191217_091843_driversxallATTR                    24
20191217_092018_activitiesxdriversxallATTR         30
20191217_092043_ECGexpectxallATTR                  70
20191217_091942_activitiesximportancexallATTR      90
Name: Variable, dtype: int64

In [28]:
frame_Accept001_usage = frame_Accept001[frame_Accept001.analysisloc==
                                        '20191216_134827_activitiesximportancexdevice']
print(frame_Accept001_usage.groupby(['Variable','Label']).interp.count())
frame_Accept001_usage[['Label','interp']]#.interp


Variable                 Label                                                                                                                                                                                                                                                           
qxactivitiesxrecency_1   Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps    6
qxactivitiesxrecency_10  Q_Activities_Recency. Setting up and managing notifications - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider your experience with alte      4
qxactivitiesxrecency_11  Q_Activities_Recency. Searching for something on your smartphone (from the home screen) - When was the last time you di

,Label,interp
2053,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 0 > cluster 1
2054,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 0 > cluster 2
2055,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 1 < cluster 0
2057,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 1 > cluster 2
2058,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 2 < cluster 0
2059,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps",Reject H0 in favor of Ha: cluster 2 < cluster 1
2062,"Q_Activities_Recency. Charging battery (including wired charge, wireless charge, different charge options, the accuracy of the battery indicator, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience",Reject H0 in favor of Ha: cluster 0 > cluster 1
2064,"Q_Activities_Recency. Charging battery (including wired charge, wireless charge, different charge options, the accuracy of the battery indicator, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience",Reject H0 in favor of Ha: cluster 1 < cluster 0
2072,"Q_Activities_Recency. Transferring files from smartphone to a computer - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider your experienc",Reject H0 in favor of Ha: cluster 0 > cluster 2
2075,"Q_Activities_Recency. Transferring files from smartphone to a computer - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider your experienc",Reject H0 in favor of Ha: cluster 1 > cluster 2


# Examine ATTRIBUTE Importance (varimp)

In [16]:
all_files = glob.glob(path + "*/*/*vimi*.csv")


li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['analysisloc'] = filename.split('/')[10]
    df['analysisclust'] = int(filename.split('/')[11].split('_')[-1].replace('.csv','').replace('vimi',''))
    li.append(df)

vimi_frame = pd.concat(li, axis=0, ignore_index=True)

print(vimi_frame.shape)
vimi_frame.head()

(954, 8)


,Variable,Label,imp,coeff,imp_rank,coeff_rank,analysisloc,analysisclust
0,qxactivitiesximportance_22,Q_Activities_Importance. Creating and editing a note - How important is it for you that a smartphone performs well on the activities below?,0.154973,2.084714,1.0,5.0,20191217_091942_activitiesximportancexallATTR,2
1,qxactivitiesximportance_24,Q_Activities_Importance. Asking digital assistant a question - How important is it for you that a smartphone performs well on the activities below?,0.093323,2.294013,2.0,3.0,20191217_091942_activitiesximportancexallATTR,2
2,qxactivitiesximportance_6,Q_Activities_Importance. Setting up multiple windows / split screen - How important is it for you that a smartphone performs well on the activities below?,0.091917,2.628311,3.0,1.0,20191217_091942_activitiesximportancexallATTR,2
3,qxactivitiesximportance_25,"Q_Activities_Importance. Using an AR (Augmented Reality) shopping assistant (e.g., take picture of an item and get a price comparison or add virtual furniture and appli - How important is it for you that a smartphone performs well on the activities below?",0.079830,2.603740,4.0,2.0,20191217_091942_activitiesximportancexallATTR,2
4,qxactivitiesximportance_9,"Q_Activities_Importance. Customizing settings (e.g., changing wallpaper, changing security/privacy requirements, etc.) - How important is it for you that a smartphone performs well on the activities below?",0.075095,1.434115,5.0,12.0,20191217_091942_activitiesximportancexallATTR,2


# For each analysis loc... 
## extract insights

In [106]:
loc_str = 'drivers'
aloc = vimi_frame.analysisloc[vimi_frame.analysisloc.str.contains(loc_str)].unique()[0]
zclust_sub = frame_Accept005[(frame_Accept005.analysisloc==aloc)].sort_values(['clusterA','pvalue'])
print(zclust_sub.shape)
zclust_sub = zclust_sub[zclust_sub.clusterA==2]
print(zclust_sub.Variable.str.contains('recency').sum())
print(zclust_sub[zclust_sub.Variable.str.contains('recency')][zclust_sub.stat>0].shape)
print(zclust_sub[zclust_sub.Variable.str.contains('recency')][zclust_sub.stat<0].shape)
# zclust_sub[zclust_sub.Variable.str.contains('recency')]
# print(zclust_sub[zclust_sub.Variable.str.contains('recency')][zclust_sub.stat0].Label)
# print()
# print(zclust_sub[zclust_sub.Variable.str.contains('recency')][zclust_sub.stat<0].Label)
zclust_sub[~zclust_sub.Variable.str.contains('recency')]

(154, 8)
28
(23, 8)
(5, 8)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Variable,Label,interp,clusterA,clusterB,stat,pvalue,analysisloc
1392,qxcurrentxos_2,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings. Android 7.1",Reject H0 in favor of Ha: cluster 2 > cluster 0,2,0,6.027140,1.938522e-09,20191217_092018_activitiesxdriversxallATTR
1374,qxcurrentxos_5,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings. Android 9.0",Reject H0 in favor of Ha: cluster 2 < cluster 0,2,0,-5.364916,9.015809e-08,20191217_092018_activitiesxdriversxallATTR
1411,qxcurrentxos_3,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings. Android 8.0",Reject H0 in favor of Ha: cluster 2 > cluster 1,2,1,5.066337,4.434044e-07,20191217_092018_activitiesxdriversxallATTR
1410,qxcurrentxos_3,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings. Android 8.0",Reject H0 in favor of Ha: cluster 2 > cluster 0,2,0,4.726332,2.414165e-06,20191217_092018_activitiesxdriversxallATTR
1428,qxcurrentxos_8,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings. iOS 12",Reject H0 in favor of Ha: cluster 2 < cluster 0,2,0,-4.401863,1.127726e-05,20191217_092018_activitiesxdriversxallATTR
1366,qxcurrentxos_99,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings. Don’t know",Reject H0 in favor of Ha: cluster 2 < cluster 1,2,1,-4.275168,2.040952e-05,20191217_092018_activitiesxdriversxallATTR
1068,d3_4,D3. Not currently employed or in school - Which of the following best describes your current employment status?,Reject H0 in favor of Ha: cluster 2 < cluster 0,2,0,-4.052950,5.250365e-05,20191217_092018_activitiesxdriversxallATTR
1528,d1_1,D1. Are you? Male,Reject H0 in favor of Ha: cluster 2 > cluster 1,2,1,3.929056,8.883351e-05,20191217_092018_activitiesxdriversxallATTR
1492,d4_1,D4. What is the highest level of education you have completed? Some high school or less,Reject H0 in favor of Ha: cluster 2 > cluster 1,2,1,3.722432,2.023352e-04,20191217_092018_activitiesxdriversxallATTR
1537,d1_2,D1. Are you? Female,Reject H0 in favor of Ha: cluster 2 < cluster 1,2,1,-3.688826,2.327186e-04,20191217_092018_activitiesxdriversxallATTR


In [107]:
c = 1
print((vimi_frame[(vimi_frame.analysisloc==aloc)&(vimi_frame.analysisclust==c)].coeff>0).sum())
print((vimi_frame[(vimi_frame.analysisloc==aloc)&(vimi_frame.analysisclust==c)].coeff<0).sum())
vimi_frame[(vimi_frame.analysisloc==aloc)&(vimi_frame.analysisclust==c)&(vimi_frame.coeff<0)]

10
91


,Variable,Label,imp,coeff,imp_rank,coeff_rank,analysisloc,analysisclust
188,qxactivitiesxdrivers_5_15,"Q_Activities_Drivers. Taking a picture with the rear-facing camera - My [MODEL] has a beautiful interface (menu, icons, etc.) - How much do you agree or disagree with the following statements for the activities below? For each of these activities, please t",0.100943,-0.441068,1.0,13.0,20191217_092018_activitiesxdriversxallATTR,1
189,qxactivitiesxdrivers_16_16,"Q_Activities_Drivers. Taking a picture (selfie) with the front-facing camera - My [MODEL] helps me to express my true self - How much do you agree or disagree with the following statements for the activities below? For each of these activities, please thin",0.056945,-0.681696,2.0,1.0,20191217_092018_activitiesxdriversxallATTR,1
190,qxactivitiesxdrivers_23_15,"Q_Activities_Drivers. Taking a picture with the rear-facing camera - [MODEL] enables me to do what I want, how I want, and as much as I want to do with a smartphone - How much do you agree or disagree with the following statements for the activities below?",0.045900,-0.453203,3.0,12.0,20191217_092018_activitiesxdriversxallATTR,1
191,qxactivitiesxdrivers_20_9,"Q_Activities_Drivers. Customizing settings (e.g., changing wallpaper, changing security/privacy requirements, etc.) - My [MODEL] is fun to use - How much do you agree or disagree with the following statements for the activities below? For each of these act",0.044866,-0.201796,4.0,58.0,20191217_092018_activitiesxdriversxallATTR,1
192,qxactivitiesxdrivers_2_14,"Q_Activities_Drivers. Sending and receiving a text message - When I use my [MODEL], I feel that my privacy is being respected and protected - How much do you agree or disagree with the following statements for the activities below? For each of these activi",0.043853,-0.258657,5.0,40.0,20191217_092018_activitiesxdriversxallATTR,1
193,qxactivitiesxdrivers_11_14,Q_Activities_Drivers. Sending and receiving a text message - Design and capabilities of my [MODEL] make me feel like my unique identity is respected - How much do you agree or disagree with the following statements for the activities below? For each of the,0.041879,-0.268975,6.0,38.0,20191217_092018_activitiesxdriversxallATTR,1
194,qxactivitiesxdrivers_10_15,Q_Activities_Drivers. Taking a picture with the rear-facing camera - My [MODEL] empowers me to do things that I couldn't do before I purchased/started using it - How much do you agree or disagree with the following statements for the activities below? For,0.040730,-0.592282,7.0,3.0,20191217_092018_activitiesxdriversxallATTR,1
195,qxactivitiesxdrivers_2_20,"Q_Activities_Drivers. Using GPS or location services (for example, maps) - When I use my [MODEL], I feel that my privacy is being respected and protected - How much do you agree or disagree with the following statements for the activities below? For each o",0.039254,-0.208311,8.0,54.0,20191217_092018_activitiesxdriversxallATTR,1
196,qxactivitiesxdrivers_16_15,"Q_Activities_Drivers. Taking a picture with the rear-facing camera - My [MODEL] helps me to express my true self - How much do you agree or disagree with the following statements for the activities below? For each of these activities, please think of your",0.037615,-0.393085,9.0,18.0,20191217_092018_activitiesxdriversxallATTR,1
197,qxactivitiesxdrivers_18_17,"Q_Activities_Drivers. Recording a video - My [MODEL] adds the elements of surprise and delight to my life - How much do you agree or disagree with the following statements for the activities below? For each of these activities, please think of your experie",0.037579,-0.040223,10.0,94.0,20191217_092018_activitiesxdriversxallATTR,1


# The most important usage questions determining whether a user will be in cluster 0 versus cluster 1 or 2 is

In [20]:
vimi_frame[(vimi_frame.analysisloc.str.contains('usage'))&(vimi_frame.analysisclust==0)]

,Variable,Label,imp,coeff,imp_rank,coeff_rank,analysisloc,analysisclust
89,qxcurrentxos,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings.",0.865176,4.959314,1.0,1.0,20191216_134821_usagemetricsxdevice,0
90,qxcurrentxstorage,"Q_Current_Storage. What is your [MODEL]’s storage capacity? Please specify the internal storage only, do not include the external storage on SD card.",0.091052,0.186615,2.0,2.0,20191216_134821_usagemetricsxdevice,0
91,qxunlocking,Q_Unlocking. Which method do you currently use to unlock your [MODEL]?,0.015388,0.034070,3.0,6.0,20191216_134821_usagemetricsxdevice,0
92,qxtransition_1,"QxTransition_1. How easy was it to transfer settings, contacts and content (pictures, music, videos, etc.) from your old smartphone to the new one?",0.010522,-0.059047,4.0,5.0,20191216_134821_usagemetricsxdevice,0
93,qxtime,Q_Time. How long have you been using your [MODEL]?,0.007024,0.005963,5.0,7.0,20191216_134821_usagemetricsxdevice,0
94,qxcurrentxcarrier,Q_Current_Carrier. Which mobile carrier do you use on your [MODEL]?,0.006177,0.067933,6.0,3.0,20191216_134821_usagemetricsxdevice,0
95,qxpreviousxbrand,Q_Previous_Brand. Which smartphone brand did you use before [MODEL]?,0.004661,0.059531,7.0,4.0,20191216_134821_usagemetricsxdevice,0
572,qxcurrentxos,"Q_Current_OS. Which version of [SHOW IF BRAND=APPLE: iOS; SHOW IF BRAND = SAMSUNG, LG, MOTOROLA, GOOGLE: Android] are you currently using on your smartphone? If you are not sure, you can check this in your settings.",0.865176,4.959314,1.0,1.0,20191214_143445_usagemetricsxdemog,0
573,qxcurrentxstorage,"Q_Current_Storage. What is your [MODEL]’s storage capacity? Please specify the internal storage only, do not include the external storage on SD card.",0.091052,0.186615,2.0,2.0,20191214_143445_usagemetricsxdemog,0
574,qxunlocking,Q_Unlocking. Which method do you currently use to unlock your [MODEL]?,0.015388,0.034070,3.0,6.0,20191214_143445_usagemetricsxdemog,0


# The most important activities importance questions determining whether a user will be in cluster 1 versus cluster 0 or 2 is

In [21]:
vimi_frame[(vimi_frame.analysisloc.str.contains('activitiesximportance'))&(vimi_frame.analysisclust==1)]

,Variable,Label,imp,coeff,imp_rank,coeff_rank,analysisloc,analysisclust
619,qxactivitiesximportance_25,"Q_Activities_Importance. Using an AR (Augmented Reality) shopping assistant (e.g., take picture of an item and get a price comparison or add virtual furniture and appli - How important is it for you that a smartphone performs well on the activities below?",0.121277,-0.491632,1.0,1.0,20191216_134827_activitiesximportancexdevice,1
620,qxactivitiesximportance_6,Q_Activities_Importance. Setting up multiple windows / split screen - How important is it for you that a smartphone performs well on the activities below?,0.070377,-0.428473,2.0,2.0,20191216_134827_activitiesximportancexdevice,1
621,qxactivitiesximportance_24,Q_Activities_Importance. Asking digital assistant a question - How important is it for you that a smartphone performs well on the activities below?,0.065527,-0.308414,3.0,5.0,20191216_134827_activitiesximportancexdevice,1
622,qxactivitiesximportance_22,Q_Activities_Importance. Creating and editing a note - How important is it for you that a smartphone performs well on the activities below?,0.061392,-0.205284,4.0,7.0,20191216_134827_activitiesximportancexdevice,1
623,qxactivitiesximportance_15,Q_Activities_Importance. Taking a picture with the rear-facing camera - How important is it for you that a smartphone performs well on the activities below?,0.048027,0.347287,5.0,3.0,20191216_134827_activitiesximportancexdevice,1
624,qxactivitiesximportance_21,"Q_Activities_Importance. Tracking your wellbeing (steps, exercise, sleep, etc.) - How important is it for you that a smartphone performs well on the activities below?",0.042955,-0.265240,6.0,6.0,20191216_134827_activitiesximportancexdevice,1
625,qxactivitiesximportance_14,Q_Activities_Importance. Sending and receiving a text message - How important is it for you that a smartphone performs well on the activities below?,0.040587,0.327466,7.0,4.0,20191216_134827_activitiesximportancexdevice,1
626,qxactivitiesximportance_23,"Q_Activities_Importance. Making a mobile payment with your smartphone at a store terminal (e.g., Samsung Pay / Apple Pay / Google Pay) - How important is it for you that a smartphone performs well on the activities below?",0.040335,-0.156243,8.0,9.0,20191216_134827_activitiesximportancexdevice,1
627,qxactivitiesximportance_3,Q_Activities_Importance. Transferring files from smartphone to a computer - How important is it for you that a smartphone performs well on the activities below?,0.036899,-0.135855,9.0,11.0,20191216_134827_activitiesximportancexdevice,1
628,qxactivitiesximportance_19,Q_Activities_Importance. Sharing a photo or file - How important is it for you that a smartphone performs well on the activities below?,0.035788,0.094626,10.0,13.0,20191216_134827_activitiesximportancexdevice,1


# The most important loyalty metrics questions determining whether a user will be in cluster 0 versus cluster 1 or 2 is

In [22]:
vimi_frame[(vimi_frame.analysisloc.str.contains('loyalty'))&(vimi_frame.analysisclust==0)]

,Variable,Label,imp,coeff,imp_rank,coeff_rank,analysisloc,analysisclust
1169,qxadvocacy01_1,"Q_Advocacy01. How likely are you to recommend your [MODEL] or another [BRAND] smartphone to a family member, friend, or colleague looking to purchase a new smartphone?",0.441661,5.326333,1.0,2.0,20191216_134810_loyaltymetricsxdevice,0
1170,qxadvocacy02_1,Q_Advocacy02. How likely are you to leave a positive online review for your [MODEL]?,0.236124,5.957574,2.0,1.0,20191216_134810_loyaltymetricsxdevice,0
1171,qxenrichment_1,"Q_Enrichment. Based on your experience with your [MODEL], how likely are you to consider [BRAND] products if you decide to buy a new tablet, a laptop, or any other electronic product produced by this brand?",0.213951,5.035969,3.0,3.0,20191216_134810_loyaltymetricsxdevice,0
1172,qxretention_1,Q_Retention. How likely are you to select [BRAND] the next time you decide to upgrade/purchase a new smartphone?,0.108264,4.964800,4.0,4.0,20191216_134810_loyaltymetricsxdevice,0
1181,qxadvocacy01_1,"Q_Advocacy01. How likely are you to recommend your [MODEL] or another [BRAND] smartphone to a family member, friend, or colleague looking to purchase a new smartphone?",0.441661,5.326333,1.0,2.0,20191214_143315_loyaltymetricsxdemog,0
1182,qxadvocacy02_1,Q_Advocacy02. How likely are you to leave a positive online review for your [MODEL]?,0.236124,5.957574,2.0,1.0,20191214_143315_loyaltymetricsxdemog,0
1183,qxenrichment_1,"Q_Enrichment. Based on your experience with your [MODEL], how likely are you to consider [BRAND] products if you decide to buy a new tablet, a laptop, or any other electronic product produced by this brand?",0.213951,5.035969,3.0,3.0,20191214_143315_loyaltymetricsxdemog,0
1184,qxretention_1,Q_Retention. How likely are you to select [BRAND] the next time you decide to upgrade/purchase a new smartphone?,0.108264,4.964800,4.0,4.0,20191214_143315_loyaltymetricsxdemog,0


# The most important overall quality questions determining whether a user will be in cluster 0 versus cluster 1 or 2 is

In [23]:
vimi_frame[(vimi_frame.analysisloc.str.contains('quality'))&(vimi_frame.analysisclust==0)]

,Variable,Label,imp,coeff,imp_rank,coeff_rank,analysisloc,analysisclust
425,qxactivitiesxqualityxindicators_1_8,"Q_Activities_Quality_Indicators. Reinforces my trust in [BRAND] - Using your quick settings (e.g., toggle WiFi, Airplane mode, brightness, volume, flashlight) - How much do you agree or disagree with the following about different activities you do with [MO",0.036483,-0.177055,1.0,6.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
426,qxactivitiesxqualityxindicators_1_15,"Q_Activities_Quality_Indicators. Reinforces my trust in [BRAND] - Taking a picture with the rear-facing camera - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience wit",0.029249,-0.055918,2.0,68.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
427,qxactivitiesxqualityxindicators_2_12,"Q_Activities_Quality_Indicators. Is enjoyable - Scheduling a calendar event / setting reminders - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience with the preinstal",0.025901,-0.067462,3.0,59.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
428,qxactivitiesxqualityxindicators_3_9,"Q_Activities_Quality_Indicators. Meets my needs / provides the desirable result - Customizing settings (e.g., changing wallpaper, changing security/privacy requirements, etc.) - How much do you agree or disagree with the following about different activitie",0.025089,-0.067302,4.0,61.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
429,qxactivitiesxqualityxindicators_1_19,"Q_Activities_Quality_Indicators. Reinforces my trust in [BRAND] - Sharing a photo or file - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience with the preinstalled ma",0.023911,-0.076290,5.0,51.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
430,qxactivitiesxqualityxindicators_1_17,"Q_Activities_Quality_Indicators. Reinforces my trust in [BRAND] - Recording a video - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience with the preinstalled manufact",0.021510,0.085280,6.0,44.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
431,qxactivitiesxqualityxindicators_4_20,"Q_Activities_Quality_Indicators. Is simple and easy - Using GPS or location services (for example, maps) - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience with the",0.021120,0.023344,7.0,87.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
432,qxactivitiesxqualityxindicators_4_17,"Q_Activities_Quality_Indicators. Is simple and easy - Recording a video - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience with the preinstalled manufacturer apps on",0.020780,0.043284,8.0,75.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
433,qxactivitiesxqualityxindicators_4_15,"Q_Activities_Quality_Indicators. Is simple and easy - Taking a picture with the rear-facing camera - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of your experience with the preins",0.019273,0.147804,9.0,12.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0
434,qxactivitiesxqualityxindicators_3_15,"Q_Activities_Quality_Indicators. Meets my needs / provides the desirable result - Taking a picture with the rear-facing camera - How much do you agree or disagree with the following about different activities you do with [MODEL]? Again, please think of you",0.018946,-0.008371,10.0,98.0,20191214_144234_activitiesxqualityxindicatorsxdemog,0


In [ ]:
# Most